### 思路
1. 登录
2. 遍历课表ids = \[ 5000 - 6800 \]
3. 提取每个课表中的课程td
4. 保存为文件

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import bs4
import time
import random

session = requests.Session()
login_url = 'https://pass.neu.edu.cn/tpass/login' 

# 1. 获取登录页面的动态参数
login_page = session.get(login_url)
soup = BeautifulSoup(login_page.text, 'html.parser')

# 提取关键参数（东北大学登录页面的实际参数名称）
lt = soup.find('input', {'name': 'lt'})['value']
execution = soup.find('input', {'name': 'execution'})['value']
_eventId = soup.find('input', {'name': '_eventId'})['value']

# 2. 构造登录数据
username = '学号'
password = '密码'

# 暂时假设明文传输
login_data = {
    'rsa': f'{username}{password}', 
    'ul': len(username),
    'pl': len(password),
    'lt': lt,
    'execution': execution,
    '_eventId': _eventId
}

# 3. 提交登录请求
response = session.post(
    login_url,
    data=login_data,
    headers={'Referer': login_url}
)

# 4. 验证登录是否成功
if "一号通门户" in response.text:
    print("登录成功")
    # 继续爬取课表数据...
else:
    print("登录失败，检查参数或密码加密逻辑")
    print(response.text)

登录成功


In [ ]:
# 打印爬取到的html信息，简单验证是否成功获取
# class_table_url = http://219.216.96.4/eams/studentPublicScheduleQuery!courseTable.action?setting.kind=class&setting.forSemester=1&semester.id=111&ids=6600
# 
ids = 6600
params = {
    "setting.kind": "class",
    "setting.forSemester": 1,
    "semester.id": 111,
    "ids":ids
}

timetable_response = session.post("http://219.216.96.4/eams/homeExt.action")
timetable_response = session.post("http://219.216.96.4/eams/studentPublicScheduleQuery!courseTable.action", params = params)
print(timetable_response.url)
print(timetable_response.text[3000:])

http://219.216.96.4/eams/studentPublicScheduleQuery!courseTable.action?setting.kind=class&setting.forSemester=1&semester.id=111&ids=6600


In [284]:
# 检验是否可以成功解析html数据

html = timetable_response.text
# 解析HTML
soup = BeautifulSoup(html, 'html.parser')

header_row = soup.find('tr', {'align': 'center', 'class': 'gridhead'})
headers = [th.get_text(strip=True) for th in header_row.find_all('th')]
print(headers[1:])

rows = soup.find_all('tr', {'style': 'text-align: center'})
# 遍历每个<tr>标签并提取<td>标签的内容
# print(rows)
for row in rows:
    cells = [cell.get_text(strip=True) for cell in row.find_all('td')]
    print(cells)

['课程代码', '课程序号', '教师姓名', '课程名称', '开课院系', '周课时', '学分', '授课语言', '起始周', '上课人数', '是否排课', '备注']
['1', 'A0705000010', 'A087202', 'Zhihong Huang', '力学（英）', '医学与生物信息工程学院（原中荷生物医学与信息工程学院）', '21.5', '4', '外语', '11', '117', '已安排', '[11-13]', '星期日', '[1-4]', '生命B201', '']
['2', 'A0705000010', 'A087202', 'Zhihong Huang', '力学（英）', '医学与生物信息工程学院（原中荷生物医学与信息工程学院）', '21.5', '4', '外语', '11', '117', '已安排', '[11-13]', '星期三', '[5-8]', '生命B201', '']
['3', 'A0705000010', 'A087202', 'Zhihong Huang', '力学（英）', '医学与生物信息工程学院（原中荷生物医学与信息工程学院）', '21.5', '4', '外语', '11', '117', '已安排', '[11-13]', '星期一', '[7-8]', '生命B201', '']
['4', 'A0705000010', 'A087202', 'Zhihong Huang', '力学（英）', '医学与生物信息工程学院（原中荷生物医学与信息工程学院）', '21.5', '4', '外语', '11', '117', '已安排', '[11-13]', '星期一', '[3-4]', '生命B201', '']
['5', 'A0705000010', 'A087202', 'Zhihong Huang', '力学（英）', '医学与生物信息工程学院（原中荷生物医学与信息工程学院）', '21.5', '4', '外语', '11', '117', '已安排', '[12-13]', '星期五', '[9-10]', '生命B201', '']
['6', 'A0705000010', 'A087202', 'Zhihong Huang', '力学（英）', '医学与生

In [ ]:
# 根据课表ID爬取对应的html界面
def ids2html(ids):
    params = {
        "setting.kind": "class",
        "setting.forSemester": 1,
        "semester.id": 111,
        "ids":ids
    }

    timetable_response = session.post("http://219.216.96.4/eams/studentPublicScheduleQuery!courseTable.action", params = params)
    html = timetable_response.text
    # print(html)
    return html

# 将课表的html界面解析为list数据
def html2list(html):
    soup = BeautifulSoup(html, 'html.parser')
    rows = soup.find_all('tr', {'style': 'text-align: center'})
    data_rows = []
    for row in rows:
        cells = [cell.get_text(strip=True) for cell in row.find_all('td')]
        # print(len(cells))
        if cells:
            data_rows.append(cells)

    return data_rows

# 保存为excel
def save_to_excel(data_rows, headers, filename='output_with_header.xlsx'):
    df = pd.DataFrame(data_rows, columns=headers)
    
    # 使用to_excel方法保存为Excel文件
    df.to_excel(filename, index=False)  # index=False表示不保存行索引

    print("已成功保存", len(data_rows), "条课程数据")

def get_header(html):
    soup = BeautifulSoup(html, 'html.parser')

    header_rows = soup.find_all('tr', {'align': 'center', 'class': 'gridhead'})
    # print(len(header_rows))

    header_row = header_rows[-1]
    headers = [th.get_text(strip=True) for th in header_row.find_all('th')]
    # print(headers)
    # print(len(headers))
    return headers
    
# 获取标题列
html = ids2html(6600)
header = get_header(html)

# 若第一次运行，需要声明data_rows
# 分批爬取建议注释掉
# data_rows = []

# 循环爬取课表，2025年春季学期课表ids约为 5000 - 6800
for ids in range(6610, 6810, 3):
    # 获取对应班级的课表数据
    html = ids2html(ids)
    #获取数据列
    datas = html2list(html)
    data_rows.extend(datas)
    if len(datas) > 0 and len(datas[0]) == 17:  # 检验是否爬取成功
        print("课表ID:", ids, "爬取成功")
    else:
        print("课表ID:", ids, "爬取失败")
    
    # 延时处理，绕过反爬机制
    time.sleep(random.uniform(1, 5))
    
# 保存到文件夹
save_to_excel(data_rows, header, filename='output_with_header3.xlsx')

课表ID: 6610 爬取失败
课表ID: 6613 爬取失败
课表ID: 6616 爬取失败
课表ID: 6619 爬取成功
课表ID: 6622 爬取成功
课表ID: 6625 爬取成功
课表ID: 6628 爬取成功
课表ID: 6631 爬取成功
课表ID: 6634 爬取成功
课表ID: 6637 爬取失败
课表ID: 6640 爬取失败
课表ID: 6643 爬取成功
课表ID: 6646 爬取失败
课表ID: 6649 爬取成功
课表ID: 6652 爬取失败
课表ID: 6655 爬取成功
课表ID: 6658 爬取失败
课表ID: 6661 爬取成功
课表ID: 6664 爬取成功
课表ID: 6667 爬取失败
课表ID: 6670 爬取失败
课表ID: 6673 爬取失败
课表ID: 6676 爬取失败
课表ID: 6679 爬取失败
课表ID: 6682 爬取失败
课表ID: 6685 爬取失败
课表ID: 6688 爬取失败
课表ID: 6691 爬取失败
课表ID: 6694 爬取失败
课表ID: 6697 爬取失败
课表ID: 6700 爬取失败
课表ID: 6703 爬取失败
课表ID: 6706 爬取失败
课表ID: 6709 爬取失败
课表ID: 6712 爬取失败
课表ID: 6715 爬取失败
课表ID: 6718 爬取失败
课表ID: 6721 爬取失败
课表ID: 6724 爬取失败
课表ID: 6727 爬取失败
课表ID: 6730 爬取失败
课表ID: 6733 爬取失败
课表ID: 6736 爬取失败
课表ID: 6739 爬取失败
课表ID: 6742 爬取失败
课表ID: 6745 爬取失败
课表ID: 6748 爬取失败
课表ID: 6751 爬取失败
课表ID: 6754 爬取失败
课表ID: 6757 爬取失败
课表ID: 6760 爬取失败
课表ID: 6763 爬取失败
课表ID: 6766 爬取失败
课表ID: 6769 爬取失败
课表ID: 6772 爬取失败
课表ID: 6775 爬取失败
课表ID: 6778 爬取失败
课表ID: 6781 爬取失败
课表ID: 6784 爬取失败
课表ID: 6787 爬取失败
课表ID: 6790 爬取失败
课表ID: 6793 爬取失败
课表ID: 67

In [314]:
print("总共爬取到的课程条数为："len(data_rows))
# save_to_excel(data_rows, header, filename='output_with_header.xlsx')

6836


In [ ]:
# 去重并保存
def remove_duplicates(data_rows):
    # 使用集合去重，首先将内部列表转换成元组（因为列表不可哈希）
    unique_rows = set(tuple(row) for row in data_rows)
    
    # 将元组再转回列表
    unique_data_rows = [list(row) for row in unique_rows]
    
    return unique_data_rows

unique_data_rows = remove_duplicates(data_rows)
print("去重后的数据条数为：", len(unique_data_rows))

# 保存
save_to_excel(unique_data_rows, header, filename='课表.xlsx')

去重后的数据条数为： 6010
已成功保存 6010 条课程数据
